In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import scipy as sp
from pydataset import data
from env import get_db_url, user, password, host
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

import acquire as acq
#import prepare as pre
import os
directory = os.getcwd()

## Using the Titanic dataset

#### 1. Use the function defined in `acquire.py` to load the Titanic data.

#### 2. Use the function defined in `prepare.py` to prepare the titanic data.

In [2]:
titanic = acq.get_titanic_data()
titanic.head()

,Unnamed: 0,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


In [3]:
titanic = acq.prep_titanic(acq.get_titanic_data())
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embark_town,alone
0,0,3,male,22.0,1,0,7.2500,Southampton,0
1,1,1,female,38.0,1,0,71.2833,Cherbourg,0
2,1,3,female,26.0,0,0,7.9250,Southampton,1
3,1,1,female,35.0,1,0,53.1000,Southampton,0
4,0,3,male,35.0,0,0,8.0500,Southampton,1


In [4]:
train, val, test = acq.split_data(titanic,'titanic')
train.head()

,survived,pclass,sex,age,sibsp,parch,fare,embark_town,alone
829,1,1,female,62.0,0,0,80.000,Southampton,1
463,0,2,male,48.0,0,0,13.000,Southampton,1
228,0,2,male,18.0,0,0,13.000,Southampton,1
374,0,3,female,3.0,3,1,21.075,Southampton,0
494,0,3,male,21.0,0,0,8.050,Southampton,1


#### 3. Encode the categorical columns on train dataset. Create dummy variables of the categorical columns and concatenate them onto the dataframe. Remove the columns they are replacing. Repeat on validate and test.

In [5]:
train.loc[:, 'is_female'] = (train.sex == 'female').astype(int)

In [6]:
train.head()

,survived,pclass,sex,age,sibsp,parch,fare,embark_town,alone,is_female
829,1,1,female,62.0,0,0,80.000,Southampton,1,1
463,0,2,male,48.0,0,0,13.000,Southampton,1,0
228,0,2,male,18.0,0,0,13.000,Southampton,1,0
374,0,3,female,3.0,3,1,21.075,Southampton,0,1
494,0,3,male,21.0,0,0,8.050,Southampton,1,0


In [7]:
train[['is_queenstown', 'is_southampton']] = pd.get_dummies(train.embark_town,
               drop_first=True).astype(int).values
train.head()

,survived,pclass,sex,age,sibsp,parch,fare,embark_town,alone,is_female,is_queenstown,is_southampton
829,1,1,female,62.0,0,0,80.000,Southampton,1,1,0,1
463,0,2,male,48.0,0,0,13.000,Southampton,1,0,0,1
228,0,2,male,18.0,0,0,13.000,Southampton,1,0,0,1
374,0,3,female,3.0,3,1,21.075,Southampton,0,1,0,1
494,0,3,male,21.0,0,0,8.050,Southampton,1,0,0,1


In [8]:
train = train.drop(columns = 'embark_town')

In [9]:
train.head()

,survived,pclass,sex,age,sibsp,parch,fare,alone,is_female,is_queenstown,is_southampton
829,1,1,female,62.0,0,0,80.000,1,1,0,1
463,0,2,male,48.0,0,0,13.000,1,0,0,1
228,0,2,male,18.0,0,0,13.000,1,0,0,1
374,0,3,female,3.0,3,1,21.075,0,1,0,1
494,0,3,male,21.0,0,0,8.050,1,0,0,1


In [10]:
continuous_features = ['age', 'fare_x_pass']
scaler = MinMaxScaler()

#### 4. Create a function named `preprocess_titanic` that accepts the train, validate, and test titanic data, and returns the dataframes ready for modeling.

In [11]:
train, val, test = acq.split_data(titanic, 'titanic')
train.head()

,survived,pclass,sex,age,sibsp,parch,fare,embark_town,alone
829,1,1,female,62.0,0,0,80.000,Southampton,1
463,0,2,male,48.0,0,0,13.000,Southampton,1
228,0,2,male,18.0,0,0,13.000,Southampton,1
374,0,3,female,3.0,3,1,21.075,Southampton,0
494,0,3,male,21.0,0,0,8.050,Southampton,1


In [15]:
def preprocess_titanic(train, val, test, continuous_features=['age', 'fare']):
    '''
    preprocess titanic will encode any categorical features
    and proceed forward with using a minmax scaler to transform 
    continuous variables.
    
    return: three dataframes, train, validate, and test, preprocessed for modeling
    '''
    # iterate through our three dataframes
    for df in [train, val, test]:
        # assign out the encoded categoricals for embark_town
        # in the same way on all three datasets
        df[['is_queenstown', 'is_southampton']] = pd.get_dummies(df.embark_town,
               drop_first=True).astype(int).values
        # added a column that divides fare to the total in tickets bough in that transaction
        df['fare_x_pass'] = df['fare'] / (df['sibsp'] + df['parch'] + df['alone']).astype(float)
    # create a single scaler object
    scaler = MinMaxScaler()
    # fit the single scaler just once to train
    scaler.fit(train[continuous_features])
    # apply that transformation to all three data sets
    # using the same syntax, which conconates
    # '_scaled' to each feature name that was fed in
    for df in [train, val, test]:
        df[[continuous_features[0]+'_scaled', continuous_features[1]+'_scaled']] = \
        scaler.transform(df[continuous_features])
    for df in [train, val, test]:
        df['is_female'] = np.where(df['sex'] == 'female', 1, 0)
    preprocessed_dfs = []
    for df in [train, val, test]:
        preprocessed_dfs.append(df.drop(columns=['sex', 'age', 'fare', 'embark_town']))
    return preprocessed_dfs

In [16]:
train_processed, val_processed, test_processed = preprocess_titanic(train, val, test)

In [17]:
train_processed.head()

,survived,pclass,sibsp,parch,alone,is_queenstown,is_southampton,age_scaled,fare_scaled,is_female,fare_x_pass
829,1,1,0,0,1,0,1,0.773813,0.156150,1,80.00000
463,0,2,0,0,1,0,1,0.597889,0.025374,0,13.00000
228,0,2,0,0,1,0,1,0.220910,0.025374,0,13.00000
374,0,3,3,1,0,0,1,0.032420,0.041136,1,5.26875
494,0,3,0,0,1,0,1,0.258608,0.015713,0,8.05000


## Using the Telco dataset

#### 1. Use the function defined in `acquire.py` to load the Telco data.

In [18]:
telco = acq.get_telco_data()
telco.head()

,Unnamed: 0,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,0,2,1,2,0002-ORFBO,Female,0,Yes,Yes,9,...,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check
1,1,2,1,1,0003-MKNFE,Male,0,No,No,9,...,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
2,2,1,2,1,0004-TLHLJ,Male,0,No,No,4,...,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,3,1,2,1,0011-IGKFF,Male,1,Yes,No,13,...,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,4,2,2,1,0013-EXCHZ,Female,1,Yes,No,3,...,Yes,Yes,No,Yes,83.9,267.4,Yes,Month-to-month,Fiber optic,Mailed check


#### 2. Use the function defined in `prepare.py` to prepare the Telco data.

In [19]:
telco = acq.prep_telco(telco)
telco.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
customer_id,,,,,,,,,,,,,,,,,,,,
0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.30,No,One year,DSL,Mailed check
0003-MKNFE,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.40,No,Month-to-month,DSL,Mailed check
0004-TLHLJ,Male,0,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
0011-IGKFF,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
0013-EXCHZ,Female,1,Yes,No,3,Yes,No,No,No,No,Yes,Yes,No,Yes,83.9,267.40,Yes,Month-to-month,Fiber optic,Mailed check


In [20]:
train, val, test = acq.split_data(telco, 'telco')
train.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
customer_id,,,,,,,,,,,,,,,,,,,,
7439-DKZTW,Male,0,No,No,1,Yes,No,No,No,No,No,Yes,No,Yes,80.55,80.55,No,Month-to-month,Fiber optic,Electronic check
0557-ASKVU,Female,0,Yes,Yes,18,Yes,No,No,No,Yes,Yes,No,No,Yes,54.40,957.10,No,One year,DSL,Credit card (automatic)
7255-SSFBC,Male,0,Yes,Yes,72,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,112.25,8041.65,No,Two year,Fiber optic,Bank transfer (automatic)
0311-QYWSS,Female,0,No,No,6,Yes,No,Yes,No,No,No,No,No,Yes,49.45,314.60,No,Month-to-month,DSL,Electronic check
1926-QUZNN,Female,0,Yes,No,72,Yes,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Yes,25.25,1841.20,No,Two year,None,Bank transfer (automatic)


In [ ]:
#train.loc[:, 'Month-to-month'] = (train.contract_type == 'Month-to-month').astype(int)
#train.head()

In [21]:
train[['One_year_con', 'Two_year_con']] = pd.get_dummies(train.contract_type,
               drop_first=True).astype(int).values
train.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,...,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,One_year_con,Two_year_con
customer_id,,,,,,,,,,,,,,,,,,,,,
7439-DKZTW,Male,0,No,No,1,Yes,No,No,No,No,...,No,Yes,80.55,80.55,No,Month-to-month,Fiber optic,Electronic check,0,0
0557-ASKVU,Female,0,Yes,Yes,18,Yes,No,No,No,Yes,...,No,Yes,54.40,957.10,No,One year,DSL,Credit card (automatic),1,0
7255-SSFBC,Male,0,Yes,Yes,72,Yes,Yes,No,Yes,Yes,...,Yes,Yes,112.25,8041.65,No,Two year,Fiber optic,Bank transfer (automatic),0,1
0311-QYWSS,Female,0,No,No,6,Yes,No,Yes,No,No,...,No,Yes,49.45,314.60,No,Month-to-month,DSL,Electronic check,0,0
1926-QUZNN,Female,0,Yes,No,72,Yes,Yes,No internet service,No internet service,No internet service,...,No internet service,Yes,25.25,1841.20,No,Two year,None,Bank transfer (automatic),0,1


In [22]:
train.payment_type.value_counts()

Electronic check             1407
Mailed check                  946
Credit card (automatic)       937
Bank transfer (automatic)     935
Name: payment_type, dtype: int64

In [23]:
train.loc[:, 'has_phone_service'] = (train.phone_service == 'Yes').astype(int)
train.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,...,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,One_year_con,Two_year_con,has_phone_service
customer_id,,,,,,,,,,,,,,,,,,,,,
7439-DKZTW,Male,0,No,No,1,Yes,No,No,No,No,...,Yes,80.55,80.55,No,Month-to-month,Fiber optic,Electronic check,0,0,1
0557-ASKVU,Female,0,Yes,Yes,18,Yes,No,No,No,Yes,...,Yes,54.40,957.10,No,One year,DSL,Credit card (automatic),1,0,1
7255-SSFBC,Male,0,Yes,Yes,72,Yes,Yes,No,Yes,Yes,...,Yes,112.25,8041.65,No,Two year,Fiber optic,Bank transfer (automatic),0,1,1
0311-QYWSS,Female,0,No,No,6,Yes,No,Yes,No,No,...,Yes,49.45,314.60,No,Month-to-month,DSL,Electronic check,0,0,1
1926-QUZNN,Female,0,Yes,No,72,Yes,Yes,No internet service,No internet service,No internet service,...,Yes,25.25,1841.20,No,Two year,None,Bank transfer (automatic),0,1,1


In [24]:
train.loc[:, 'has_no_internet'] = (train.internet_service_type == 'None').astype(int)
train.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,...,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,One_year_con,Two_year_con,has_phone_service,has_no_internet
customer_id,,,,,,,,,,,,,,,,,,,,,
7439-DKZTW,Male,0,No,No,1,Yes,No,No,No,No,...,80.55,80.55,No,Month-to-month,Fiber optic,Electronic check,0,0,1,0
0557-ASKVU,Female,0,Yes,Yes,18,Yes,No,No,No,Yes,...,54.40,957.10,No,One year,DSL,Credit card (automatic),1,0,1,0
7255-SSFBC,Male,0,Yes,Yes,72,Yes,Yes,No,Yes,Yes,...,112.25,8041.65,No,Two year,Fiber optic,Bank transfer (automatic),0,1,1,0
0311-QYWSS,Female,0,No,No,6,Yes,No,Yes,No,No,...,49.45,314.60,No,Month-to-month,DSL,Electronic check,0,0,1,0
1926-QUZNN,Female,0,Yes,No,72,Yes,Yes,No internet service,No internet service,No internet service,...,25.25,1841.20,No,Two year,None,Bank transfer (automatic),0,1,1,1


In [25]:
train[['Credit_card', 'Electronic_check', 'Mailed_check']] = pd.get_dummies(train.payment_type,
               drop_first=True).astype(int).values
train.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,...,contract_type,internet_service_type,payment_type,One_year_con,Two_year_con,has_phone_service,has_no_internet,Credit_card,Electronic_check,Mailed_check
customer_id,,,,,,,,,,,,,,,,,,,,,
7439-DKZTW,Male,0,No,No,1,Yes,No,No,No,No,...,Month-to-month,Fiber optic,Electronic check,0,0,1,0,0,1,0
0557-ASKVU,Female,0,Yes,Yes,18,Yes,No,No,No,Yes,...,One year,DSL,Credit card (automatic),1,0,1,0,1,0,0
7255-SSFBC,Male,0,Yes,Yes,72,Yes,Yes,No,Yes,Yes,...,Two year,Fiber optic,Bank transfer (automatic),0,1,1,0,0,0,0
0311-QYWSS,Female,0,No,No,6,Yes,No,Yes,No,No,...,Month-to-month,DSL,Electronic check,0,0,1,0,0,1,0
1926-QUZNN,Female,0,Yes,No,72,Yes,Yes,No internet service,No internet service,No internet service,...,Two year,None,Bank transfer (automatic),0,1,1,1,0,0,0


#### 3. Encode the categorical columns on train.
- Encode at least one column using `.replace`
- Encode at least one column using `.map`
- Encode the rest of the columns by creating dummy variables and concatenating them onto the dataframe.


In [26]:
train.churn.map(
{'Yes': 1,
'No': 0})[:10]

customer_id
7439-DKZTW    0
0557-ASKVU    0
7255-SSFBC    0
0311-QYWSS    0
1926-QUZNN    0
5277-ZLOOR    1
1936-CZAKF    0
9526-JAWYF    1
7660-HDPJV    1
3027-YNWZU    0
Name: churn, dtype: int64

In [27]:
train[['Phone_No','Phone_Yes']] = pd.get_dummies(train.phone_service,
               drop_first=False).astype(int).values
train.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,...,payment_type,One_year_con,Two_year_con,has_phone_service,has_no_internet,Credit_card,Electronic_check,Mailed_check,Phone_No,Phone_Yes
customer_id,,,,,,,,,,,,,,,,,,,,,
7439-DKZTW,Male,0,No,No,1,Yes,No,No,No,No,...,Electronic check,0,0,1,0,0,1,0,0,1
0557-ASKVU,Female,0,Yes,Yes,18,Yes,No,No,No,Yes,...,Credit card (automatic),1,0,1,0,1,0,0,0,1
7255-SSFBC,Male,0,Yes,Yes,72,Yes,Yes,No,Yes,Yes,...,Bank transfer (automatic),0,1,1,0,0,0,0,0,1
0311-QYWSS,Female,0,No,No,6,Yes,No,Yes,No,No,...,Electronic check,0,0,1,0,0,1,0,0,1
1926-QUZNN,Female,0,Yes,No,72,Yes,Yes,No internet service,No internet service,No internet service,...,Bank transfer (automatic),0,1,1,1,0,0,0,0,1


In [ ]:
train = train.drop(columns = 'phone_service')

In [28]:
train.loc[:, 'is_churn'] = (train.churn == 'Yes').astype(int)
train.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,...,One_year_con,Two_year_con,has_phone_service,has_no_internet,Credit_card,Electronic_check,Mailed_check,Phone_No,Phone_Yes,is_churn
customer_id,,,,,,,,,,,,,,,,,,,,,
7439-DKZTW,Male,0,No,No,1,Yes,No,No,No,No,...,0,0,1,0,0,1,0,0,1,0
0557-ASKVU,Female,0,Yes,Yes,18,Yes,No,No,No,Yes,...,1,0,1,0,1,0,0,0,1,0
7255-SSFBC,Male,0,Yes,Yes,72,Yes,Yes,No,Yes,Yes,...,0,1,1,0,0,0,0,0,1,0
0311-QYWSS,Female,0,No,No,6,Yes,No,Yes,No,No,...,0,0,1,0,0,1,0,0,1,0
1926-QUZNN,Female,0,Yes,No,72,Yes,Yes,No internet service,No internet service,No internet service,...,0,1,1,1,0,0,0,0,1,0


In [31]:
train = train.drop(columns = 'phone_service')

In [29]:
train = train.drop(columns = ['churn','phone_service','contract_type', 'internet_service_type','device_protection', 'dependents', 'senior_citizen', 'online_backup', 'device_protection','online_security', 'paperless_billing','gender','partner', 'tech_support', 'streaming_tv', 'streaming_movies', 'multiple_lines', 'payment_type'])

In [32]:
train.head()

,tenure,monthly_charges,total_charges,One_year_con,Two_year_con,has_phone_service,has_no_internet,Credit_card,Electronic_check,Mailed_check,Phone_No,Phone_Yes,is_churn
customer_id,,,,,,,,,,,,,
7439-DKZTW,1,80.55,80.55,0,0,1,0,0,1,0,0,1,0
0557-ASKVU,18,54.40,957.10,1,0,1,0,1,0,0,0,1,0
7255-SSFBC,72,112.25,8041.65,0,1,1,0,0,0,0,0,1,0
0311-QYWSS,6,49.45,314.60,0,0,1,0,0,1,0,0,1,0
1926-QUZNN,72,25.25,1841.20,0,1,1,1,0,0,0,0,1,0


#### 4. Repeat the same steps on validate and test.

In [36]:
val[['One_year_con', 'Two_year_con']] = pd.get_dummies(val.contract_type,drop_first=True).astype(int).values
val.loc[:, 'has_phone_service'] = (val.phone_service == 'Yes').astype(int)
val.loc[:, 'has_no_internet'] = (val.internet_service_type == 'None').astype(int)
val[['Credit_card', 'Electronic_check', 'Mailed_check']] = pd.get_dummies(val.payment_type,drop_first=True).astype(int).values
val[['Phone_No','Phone_Yes']] = pd.get_dummies(val.phone_service,drop_first=False).astype(int).values
val.loc[:, 'is_churn'] = (val.churn == 'Yes').astype(int)

val.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,...,One_year_con,Two_year_con,has_phone_service,has_no_internet,Credit_card,Electronic_check,Mailed_check,Phone_No,Phone_Yes,is_churn
customer_id,,,,,,,,,,,,,,,,,,,,,
4760-THGOT,Female,0,Yes,No,43,Yes,Yes,Yes,Yes,No,...,0,0,1,0,0,1,0,0,1,0
4023-RTIQM,Female,1,Yes,No,31,No,No phone service,Yes,Yes,Yes,...,1,0,0,0,1,0,0,1,0,0
3407-QGWLG,Male,0,No,Yes,3,Yes,No,No internet service,No internet service,No internet service,...,0,0,1,1,0,0,1,0,1,0
5642-MHDQT,Female,0,Yes,Yes,53,Yes,No,No internet service,No internet service,No internet service,...,1,0,1,1,0,0,0,0,1,0
7080-TNUWP,Male,0,Yes,No,70,Yes,Yes,No,Yes,Yes,...,1,0,1,0,0,0,0,0,1,0


In [38]:
val = val.drop(columns = ['churn','phone_service','contract_type', 'internet_service_type','device_protection', 'dependents', 'senior_citizen', 'online_backup', 'device_protection','online_security', 'paperless_billing','gender','partner', 'tech_support', 'streaming_tv', 'streaming_movies', 'multiple_lines', 'payment_type'])
val.head()

,tenure,monthly_charges,total_charges,One_year_con,Two_year_con,has_phone_service,has_no_internet,Credit_card,Electronic_check,Mailed_check,Phone_No,Phone_Yes,is_churn
customer_id,,,,,,,,,,,,,
4760-THGOT,43,94.10,4107.30,0,0,1,0,0,1,0,0,1,0
4023-RTIQM,31,50.40,1580.10,1,0,0,0,1,0,0,1,0,0
3407-QGWLG,3,20.05,75.45,0,0,1,1,0,0,1,0,1,0
5642-MHDQT,53,19.85,1039.45,1,0,1,1,0,0,0,0,1,0
7080-TNUWP,70,95.00,6602.90,1,0,1,0,0,0,0,0,1,0


In [37]:
test[['One_year_con', 'Two_year_con']] = pd.get_dummies(test.contract_type, drop_first=True).astype(int).values
test.loc[:, 'has_phone_service'] = (test.phone_service == 'Yes').astype(int)
test.loc[:, 'has_no_internet'] = (test.internet_service_type == 'None').astype(int)
test[['Credit_card', 'Electronic_check', 'Mailed_check']] = pd.get_dummies(test.payment_type, drop_first=True).astype(int).values
test[['Phone_No','Phone_Yes']] = pd.get_dummies(test.phone_service,drop_first=False).astype(int).values
test.loc[:, 'is_churn'] = (test.churn == 'Yes').astype(int)

test.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,...,One_year_con,Two_year_con,has_phone_service,has_no_internet,Credit_card,Electronic_check,Mailed_check,Phone_No,Phone_Yes,is_churn
customer_id,,,,,,,,,,,,,,,,,,,,,
3739-YBWAB,Male,0,Yes,No,36,No,No phone service,Yes,No,Yes,...,1,0,0,0,0,0,1,1,0,0
3654-ARMGP,Female,0,No,No,61,Yes,Yes,No,Yes,Yes,...,0,1,1,0,1,0,0,0,1,0
9220-CXRSC,Female,0,Yes,Yes,69,Yes,Yes,Yes,No,No,...,0,1,1,0,1,0,0,0,1,0
7997-EASSD,Female,0,Yes,No,63,Yes,Yes,No,No,No,...,1,0,1,0,1,0,0,0,1,0
7595-EHCDL,Male,0,Yes,Yes,32,No,No phone service,No,No,Yes,...,0,0,0,0,1,0,0,1,0,0


In [39]:
test = test.drop(columns = ['churn','phone_service','contract_type', 'internet_service_type','device_protection', 'dependents', 'senior_citizen', 'online_backup', 'device_protection','online_security', 'paperless_billing','gender','partner', 'tech_support', 'streaming_tv', 'streaming_movies', 'multiple_lines', 'payment_type'])
test.head()

,tenure,monthly_charges,total_charges,One_year_con,Two_year_con,has_phone_service,has_no_internet,Credit_card,Electronic_check,Mailed_check,Phone_No,Phone_Yes,is_churn
customer_id,,,,,,,,,,,,,
3739-YBWAB,36,35.35,1317.95,1,0,0,0,0,0,1,1,0,0
3654-ARMGP,61,88.65,5321.25,0,1,1,0,1,0,0,0,1,0
9220-CXRSC,69,61.40,4059.85,0,1,1,0,1,0,0,0,1,0
7997-EASSD,63,81.20,4965.10,1,0,1,0,1,0,0,0,1,0
7595-EHCDL,32,50.60,1653.45,0,0,0,0,1,0,0,1,0,0


#### 5. Create a function named `prep_telco` that accepts the train, validate, and test telco data, and returns the dataframes ready for modeling.

In [ ]:
def preprocess_telco(train, val, test, continuous_features=['tenure', 'monthly_charges', 'total_charges']):
    
    '''
    preprocess telco will encode any categorical features
    and proceed forward with using a minmax scaler to transform 
    continuous variables.
    
    return: three dataframes, train, validate, and test, preprocessed for modeling
    '''
    # iterate through our three dataframes
    for df in [train, val, test]:
        # assign out the encoded categoricals for 
        # in the same way on all three datasets
        df[['One_year_contract', 'Two_year_contract']] = pd.get_dummies(df.contract_type,
               drop_first=True).astype(int).values
        

    
   
   
    
    df.loc[:, 'has_phone_service'] = (df.phone_service == 'Yes').astype(int)
    df.loc[:, 'has_no_internet'] = (df.internet_service_type == 'None').astype(int)
    df.loc[:, 'is_churn'] = (df.churn == 'Yes').astype(int)
    
    df.loc[:,'internet_service_type'] = df.internet_service_type.fillna('no internet')
    df = df.set_index('customer_id')
    df.loc[:,'total_charges'] = (df.total_charges + '0')
    df.total_charges = df.total_charges.astype(float)
    return df

     # create a single scaler object
    scaler = MinMaxScaler()
    # fit the single scaler just once to train
    scaler.fit(train[continuous_features])
    # apply that transformation to all three data sets
    # using the same syntax, which conconates
    # '_scaled' to each feature name that was fed in
    for df in [train, val, test]:
        df[[continuous_features[0]+'_scaled', continuous_features[1]+'_scaled']] = \
        scaler.transform(df[continuous_features])
    for df in [train, val, test]:
        df[['Churn_No','Churn_Yes']] = pd.get_dummies(df.churn, drop_first=False).astype(int).values
        df[['Credit_card', 'Electronic_check', 'Mailed_check']] = pd.get_dummies(df.payment_type,
               drop_first=True).astype(int).values
        df[['One_year_con', 'Two_year_con']] = pd.get_dummies(df.contract_type,
               drop_first=True).astype(int).values    
    preprocessed_dfs = []
    for df in [train, val, test]:
        preprocessed_dfs.append(df.drop(columns=['Unnamed: 0',
        'internet_service_type_id',
        'payment_type_id',
        'contract_type_id', 
        'churn','phone_service',
        'contract_type', 
        'internet_service_type',
        'device_protection', 
        'dependents', 'senior_citizen', 
        'online_backup', 'device_protection',
        'online_security', 'paperless_billing',
        'gender','partner', 'tech_support',
        'streaming_tv', 'streaming_movies', 
        'multiple_lines', 'payment_type']))
    return preprocessed_dfs


   
    
    
